In [1]:
!pip install -q --upgrade transformers==4.25.1 diffusers ftfy accelerate #there's probably more here if reqd for google colab but I'll work it out later

In [2]:
#@title test
from base64 import b64encode

import numpy
import torch
from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from huggingface_hub import notebook_login

from PIL import Image
import requests
from io import BytesIO
import numpy as np


# For video display:
from IPython.display import HTML
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, logging
import os

import torch.nn.functional as F
torch.manual_seed(1)
#if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == torch_device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

/mnt/c/Users/hew7/Documents/venvs/RF-demos/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

text_encoder = text_encoder.to(torch_device).half()

In [ ]:
def tokenize(stringlist):
    tokenized = tokenizer(stringlist, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    return tokenized

def detokenizer(tokenized):
    return tokenizer.decode(tokenized['input_ids'][0], skip_special_tokens=True)

def sentence_embedder(stringlist):
    inputs=tokenize(stringlist)
    inputs=inputs.to(torch_device)
    with torch.no_grad():
        sentence_embeddings=text_encoder(**inputs)
    return sentence_embeddings

uncond_embeddings=sentence_embedder([''])[0]

In [ ]:
# Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

vae = vae.to(torch_device).half()

In [ ]:
# Function for turning an image into a VAE input
def img2VAETensor(PILImage):
    PILImage=PILImage.convert('RGB')
    ImgTensor=tfms.ToTensor()(PILImage).unsqueeze(0) * 2.0 - 1.0
    return ImgTensor


In [ ]:
# VAE encoder function

def VAE_encode(VAETensor):
    init_encode=vae.encode(VAETensor)
    encoded_sample=init_encode.latent_dist.sample()*0.18215
    return encoded_sample


In [ ]:
def VAE_decode(VAE_encoded_sample):
    rescaled_sample=(1/0.18215)*VAE_encoded_sample

    with torch.no_grad():
        decodedTensor=vae.decode(rescaled_sample).sample
    decodedTensor=(decodedTensor/2+0.5).clamp(0,1)
    decodedTensor=decodedTensor.detach().cpu().permute(0, 2, 3, 1).numpy()
    decodedTensor=(decodedTensor * 255).round().astype("uint8")
    decodedTensorImg=Image.fromarray(decodedTensor[0])

    return decodedTensorImg


In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


In [ ]:
def get_image(URL):
    ImgURL = requests.get(URL)
    Img = Image.open(BytesIO(ImgURL.content))
    
    lrdiff=Img.size[0]-min(Img.size)
    tbdiff=Img.size[1]-min(Img.size)
    Img = Img.crop((0,0,Img.size[0]-lrdiff, Img.size[1]-tbdiff))
    Img = Img.resize((512,512))
    return Img

def downres(Img):
    Img=Img.resize((64,64))
    Img=Img.resize((512,512))
    return Img

def VAE_demo(URL):
    orig_img=get_image(URL)
    downres_img=downres(orig_img)

    tensor_Img=img2VAETensor(orig_img).to(torch_device).half()
    encodedSample=VAE_encode(tensor_Img)
    decoded_img=VAE_decode(encodedSample)
    return image_grid([orig_img, downres_img, decoded_img], 1, 3)

In [ ]:
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

unet=unet.to(torch_device).half()

scheduler = LMSDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

# this is setting the number of theoretical denoising steps we're doing. This number is arbitrary - ignore it for now.
scheduler.set_timesteps(100)

# this is setting the step which we're denoising from - also ignore it for now.
demo_timestep=80

# Stable Diffusion explainer!

Hello! welcome to my explainer for stable diffusion - specifically targeting a non-technical audience. This will be a quick rundown of how Latent Diffusion Models work, specifically focusing on Stable Diffusion v1.4, as it's probably the most intuitive to explain. 

This explainer will be split into multiple parts - first, we will give a high level overview of the model cascade itself, then we'll go into each component of the cascade, and finally, we'll put the model cascade together at the end!

This is the first section out of 5: a high level explanation of the model cascade. The other sections are accessible here:

1. Intro to the architecture
2. CLIP Text embedder
3. Variational AutoEncoder
4. UNet denoiser
5. Conclusion - putting it all together

*turn these into links eventually*

## Part 5: Putting it all together
Okay, I've shown you a lot of crap at this point - but how does this all fit together?

Well, let's imagine we want to create an image from scratch, based on a text prompt. We currently have loaded:

a model that can turn text into vector embeddings
a model that can denoise a latent space, and can be guided by vector embeddings
a model that can turn a latent space into an image
So, what can we do?

Well, if we try denoising an image that is completely noise, what happens?

In [ ]:
# define some functions to reuse

def init_noise(random_seed, scheduler):

    latents = torch.randn(
      (1, 4, 64,64),
        generator=torch.manual_seed(random_seed)
    )
    latents = latents.to(torch_device)
    
    latents = latents * scheduler.init_noise_sigma
    return latents

def LDM_noguide_forward(latents, timestep, scheduler=scheduler):
    latents=latents.half()
    latent_model_input = scheduler.scale_model_input(latents, scheduler.timesteps[timestep]).half()
    
    # passing the scaled latent inputs to the unet, with the timestep and the unconditional embeddings
    with torch.no_grad():
        noise_pred = unet(latent_model_input, scheduler.timesteps[timestep], encoder_hidden_states=uncond_embeddings).sample
    
    # output the clean latent
    all_latents=scheduler.step(noise_pred, scheduler.timesteps[timestep], latents)

    return all_latents

In [ ]:
#reinitialise scheduler timesteps
scheduler.set_timesteps(5)

#initialise noise
latents=init_noise(random_seed=9001,
                  scheduler=scheduler)

all_latents=LDM_noguide_forward(latents=latents,
                               timestep=0,
                               scheduler=scheduler)

#output image from VAE decoder
denoised_latents=all_latents.pred_original_sample   

denoisedDecoded=VAE_decode(denoised_latents)
denoisedDecoded

Weird! when we take completely random noise, denoise it, and then put it through the VAE, you sort of get an extremely blurry looking picture.

Now what happens if we add noise back into the picture, and then denoise it again?

In [ ]:
latents=all_latents.prev_sample
all_latents=LDM_noguide_forward(latents=latents,
                               timestep=1)

#output image from VAE decoder
denoised_latents=all_latents.pred_original_sample   
denoisedDecoded=VAE_decode(denoised_latents)
denoisedDecoded

Hmm - slightly more structure... let's do it a few more times


In [ ]:
latents=all_latents.prev_sample
all_latents=LDM_noguide_forward(latents=latents,
                               timestep=2)

#output image from VAE decoder
denoised_latents=all_latents.pred_original_sample   
denoisedDecoded=VAE_decode(denoised_latents)
denoisedDecoded

In [ ]:
latents=all_latents.prev_sample
all_latents=LDM_noguide_forward(latents=latents,
                               timestep=3)

#output image from VAE decoder
denoised_latents=all_latents.pred_original_sample   
denoisedDecoded=VAE_decode(denoised_latents)
denoisedDecoded

In [ ]:
latents=all_latents.prev_sample
all_latents=LDM_noguide_forward(latents=latents,
                               timestep=4)

#output image from VAE decoder
denoised_latents=all_latents.pred_original_sample   
denoisedDecoded=VAE_decode(denoised_latents)
denoisedDecoded

Look at that! That's almost a picture! It sort of looks like a bathroom, with a towel rack, and maybe a mirror? very impressionistic.

What's happening here is that every time the image is denoised, a teeny tiny bit of detail is assumed to exist in the leftover image (even though we know for a fact that the image is completely random garbage). The model is iteratively leaving behind bits of what it "assumes" is correct details of the image. 

You can imagine then, that two particular factors play a great role in determining what the model decides to "leave behind" at each step - the initial structure of the noise, and the number of denoising steps it takes. With the initial noise, there will be particular random features within that noise that the model will "leave behind", and with the number of iterations, think of it as the "step size" of the model being slightly smaller - the size and shape of the noise being added will be different at each step, and therefore the detail being left behind will be inherently smaller, but also different as well. 

I'll wrap the above steps into a neat little function for you to iterate over, with those two particular parameters exposed (i.e. you can change them willy nilly). Play around and see what you find!

In [ ]:
def seed_timestep_demo(timesteps, seed):

    #reinitialise scheduler timesteps
    scheduler.set_timesteps(timesteps)
    
    #initialise noise
    latents=init_noise(random_seed=seed,
                      scheduler=scheduler)
    
    
    for i in tqdm(range(timesteps)):
        all_latents=LDM_noguide_forward(latents=latents,
                                       timestep=i)
        
        #output image from VAE decoder
        denoised_latents=all_latents.pred_original_sample   
        
        denoisedDecoded=VAE_decode(denoised_latents)
        latents=all_latents.prev_sample
        denoisedDecoded
    return denoisedDecoded

In [ ]:
# ITERATE HERE
seed_timestep_demo(timesteps=50,
                  seed=2)

What I've shown you is essentially just the Latent Diffusion model by itself - but how do we add the text in? Well - remember how we put in the empty text embedding for the above exercises? For starters, lets just make that non-empty and see how we go! Have a playaround with the parameters and inputs as well if you like.

In [ ]:
def LDM_textonly_forward(latents, timestep, text_embeddings, scheduler=scheduler):
    latents=latents.half()
    latent_model_input = scheduler.scale_model_input(latents, scheduler.timesteps[timestep]).half()
    
    # passing the scaled latent inputs to the unet, with the timestep and the unconditional embeddings
    with torch.no_grad():
        noise_pred = unet(latent_model_input, scheduler.timesteps[timestep], encoder_hidden_states=text_embeddings).sample
    
    # output the clean latent
    all_latents=scheduler.step(noise_pred, scheduler.timesteps[timestep], latents)

    return all_latents

def seed_timestep_textinput_demo(timesteps, seed, text_embeddings, scheduler):

    #reinitialise scheduler timesteps
    scheduler.set_timesteps(timesteps)
    
    #initialise noise
    latents=init_noise(random_seed=seed,
                      scheduler=scheduler)
    
    
    for i in tqdm(range(timesteps)):
        all_latents=LDM_textonly_forward(latents=latents,
                                       timestep=i,
                                       text_embeddings=text_embeddings,
                                        scheduler=scheduler)
        
        #output image from VAE decoder
        denoised_latents=all_latents.pred_original_sample   
        
        denoisedDecoded=VAE_decode(denoised_latents)
        latents=all_latents.prev_sample
        denoisedDecoded
    return denoisedDecoded

In [ ]:
textinput=['Horse']

text_embeddings=sentence_embedder(textinput)[0]

seed_timestep_textinput_demo(timesteps=50,
                             seed=10,
                             text_embeddings=text_embeddings,
                             scheduler=scheduler
                            )

Hmm - interesting. It doesn't seem to be working too well does it? For the most part, it's garbage - Maaaybe it has some little features in the output that look like the text input, but for the most part, it's fairly abstract.

So how do we make it better? Well, some smart folks at Stability.AI went "hey, why don't we take the denoised output from the unconditioned diffusion model, and move the conditioned noise prediction further away from the unconditioned prediction?" This technique is called Classifier-Free Guidance - and that's what we're going to do! Let's implement that in code.

In [ ]:
def LDM_guided_forward(latents, timestep, text_embeddings, uncond_embeddings, guidance_scale=7.5, scheduler=scheduler):
    
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings]) # batch the conditioned and unconditioned text embeddings
    latents=latents.half()
    latent_model_input=torch.cat([latents] * 2) #make sure latents are the same dimensions as text embeddings
    latent_model_input = scheduler.scale_model_input(latent_model_input, scheduler.timesteps[timestep]).half()
    
    # passing the scaled latent inputs to the unet, with the timestep and the unconditional embeddings
    with torch.no_grad():
        noise_pred = unet(latent_model_input, scheduler.timesteps[timestep], encoder_hidden_states=text_embeddings).sample
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)

    # CLASSIFIER-FREE GUIDANCE
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond) 
    
    # output the clean latent
    all_latents=scheduler.step(noise_pred, scheduler.timesteps[timestep], latents)

    return all_latents

def full_demo(timesteps, seed, text_embeddings, uncond_embeddings, guidance_scale=7.5, scheduler=scheduler):

    #reinitialise scheduler timesteps
    scheduler.set_timesteps(timesteps)
    
    #initialise noise
    latents=init_noise(random_seed=seed,
                      scheduler=scheduler)
    
    
    for i in tqdm(range(timesteps)):
        all_latents=LDM_guided_forward(latents=latents,
                                       timestep=i,
                                       text_embeddings=text_embeddings,
                                       uncond_embeddings=uncond_embeddings,
                                       guidance_scale=guidance_scale,
                                       scheduler=scheduler)
        
        #output image from VAE decoder
        denoised_latents=all_latents.pred_original_sample   
        
        denoisedDecoded=VAE_decode(denoised_latents)
        latents=all_latents.prev_sample
        denoisedDecoded
    return denoisedDecoded

In [ ]:
textinput=['an astronaut on a horse']
uncondinput=['']

text_embeddings=sentence_embedder(textinput)[0]
uncond_embeddings=sentence_embedder(uncondinput)[0]

full_demo(timesteps=50, 
          seed=800, 
          text_embeddings=text_embeddings, 
          uncond_embeddings=uncond_embeddings, 
          guidance_scale=7.5, 
          scheduler=scheduler)

One thing you'll note is that we also have a guidance scale parameter. In the function written above, you can see that it multiplies the "difference" between the conditioned and unconditioned embeddings, and so you can think of it as amplifying how "conditioned" the generation is going to be. Higher values for the guidance scale should intuitively mean that your output should be more like your text input (i.e. if you put in the word "dog", you would get an image that looks more like a dog) - however, what that means to the diffusion model probably doesn't quite mean what you think! Have a play around and see what you find!

Another thing worth noting - text embeddings, like I've mentioned earlier, are just a set of numbers. You can do a bunch of arithmetic to them to transform the input embedding, which can also have funky effects. The same goes for the unconditional embeddings - what happens if you don't put an empty string in to the unconditional embeddings?

And that's it! That's the whole of stable diffusion! Hopefully this has been helpful :)